# Araştırma Komisyonları - Data Collection

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException

import pandas as pd
import time
import os
import csv

In [2]:
driver = webdriver.Chrome()
driver.implicitly_wait(30)

## Araştırma Komisyonları

- source_page: https://www.tbmm.gov.tr/Denetim/Gecmis-Donem-Meclis-Arastirma-Komisyonlari
- donem_values:
    - 22. Dönem Tüm Yasama Yılları: **221, 222, 223, 224, 225**
    - 23. Dönem Tüm Yasama Yılları: **231, 232, 234, 235**
    - 24. Dönem Tüm Yasama Yılları: **241, 242, 243, 244, 245**
    - 25. Dönem Tüm Yasama Yılları: **251, 252**
    - 26. Dönem Tüm Yasama Yılları: **261, 262, 263**
- select_donem (Dönemi ve Yasama Yılı) XPATH: //*[@id="DonemYasamaYili"]
- input_search (Sorguyu Başlat) XPATH: //*[@id="main-area"]/form/div[5]/div/input

In [3]:
def get_sonuc_page(source_page, donem):

    # Open source page
    driver.get(source_page)
    
    # Select Dönem
    select_donem = Select(driver.find_element(By.XPATH, """//*[@id="DonemYasamaYili"]"""))
    donem = str(donem)
    select_donem.select_by_value(donem)
    
    # Find "Sorguyu Başlat" button
    input_search = driver.find_element(By.XPATH, """//*[@id="main-area"]/form/div[5]/div/input""")

    # Click "Sorguyu Başlat" button
    input_search.click()
    
    # Get total page size
    pagination = driver.find_element(By.XPATH, """//*[@id="dataTable_paginate"]/ul""")
    pagination_elements = pagination.find_elements(By.TAG_NAME, "li")
    
    # if there are no records:
    if len(pagination_elements) == 2:
        page_size = 0
    # if there is one page
    elif len(pagination_elements) == 3:
        page_size = 1
    # if there are two pages:
    elif len(pagination_elements) == 4:
        page_size = 2
    # if there are more than two pages
    else:
        page_size = int(pagination_elements[-2].text)
    
    return page_size

In [4]:
def collect_urls():
    
    urls = []
    komisyon_adlari = []
    donemler = []
    
    
    # Find table
    table = driver.find_element(By.XPATH, """//*[@id="dataTable"]/tbody""")
    
    # Get rows of table
    rows = None
    while rows is None:
        try:
            rows = table.find_elements(By.TAG_NAME, "tr")
        except:
            pass
        
    for i, row in enumerate(rows):
        komisyon_adi = row.find_element(By.CLASS_NAME, "onerge-ozeti").text.split("\n")[0]
        komisyon_adlari.append(komisyon_adi)
        
        donem = row.find_elements(By.TAG_NAME, "td")[0].text.split("/")[0]
        donemler.append(donem)
        
        a_tags = row.find_elements(By.TAG_NAME, "td")[2].find_elements(By.TAG_NAME, "a")
        if len(a_tags) == 2:
            urls.append(a_tags[0].get_attribute("href"))
        else:
            urls.append("")
    return komisyon_adlari, donemler, urls

In [5]:
donemler = [221, 222, 223, 224, 225, 231, 232, 234, 235, 241, 242, 243, 244, 245, 251, 252, 261, 262, 263]
source_page = "https://www.tbmm.gov.tr/Denetim/Gecmis-Donem-Meclis-Arastirma-Komisyonlari"

In [6]:
# Create "arastirma_komisyonlari" folder if not exists
if "arastirma_komisyonlari" not in os.listdir():
    os.mkdir("arastirma_komisyonlari")

# Create csv file and headers
pd.DataFrame(columns=["komisyon", "donem", "url"]).to_csv(f"arastirma_komisyonlari/arastirma_komisyonlari.csv",
                                                   mode="w", index=False)

# Create urls folder if not exists
if "urls" not in os.listdir():
    os.mkdir("urls")

# Write urls to txt file
with open("urls/arastirma_komisyonlari_urls.txt", "a+", encoding="UTF-8") as url_file, open("arastirma_komisyonlari/arastirma_komisyonlari.csv", "a+", encoding="UTF-8") as komisyon_adi_url_file:
    for donem in donemler:
        # Get sonuc page & page size
        page_size = get_sonuc_page(source_page, donem)
        print(f"Dönem: {donem} | Page Size: {page_size} ... ", end="")
    
        page = 1
        
        while page <= page_size:
            try:
                time.sleep(1)
                # Collect urls & komisyon adlari in the page
                komisyon_adlari, donem_list, urls = collect_urls()
                for url in urls:
                    if url != "":
                        url_file.write(f"{url}\n")
                csv_writer = csv.writer(komisyon_adi_url_file)
                komisyon_zip = zip(komisyon_adlari, donem_list, urls)
                for komisyon in komisyon_zip:
                    csv_writer.writerow(komisyon)
                
                # Find "Sonraki" button
                next_button = driver.find_element(By.XPATH, """//*[@id="dataTable_next"]""")
                
                # Click "Sonraki" button
                next_button.click()
                
                page += 1
                
            except:
                time.sleep(5)
                continue
        print("Completed!")

Dönem: 221 | Page Size: 1 ... Completed!
Dönem: 222 | Page Size: 1 ... Completed!
Dönem: 223 | Page Size: 1 ... Completed!
Dönem: 224 | Page Size: 1 ... Completed!
Dönem: 225 | Page Size: 1 ... Completed!
Dönem: 231 | Page Size: 0 ... Completed!
Dönem: 232 | Page Size: 1 ... Completed!
Dönem: 234 | Page Size: 1 ... Completed!
Dönem: 235 | Page Size: 1 ... Completed!
Dönem: 241 | Page Size: 0 ... Completed!
Dönem: 242 | Page Size: 1 ... Completed!
Dönem: 243 | Page Size: 1 ... Completed!
Dönem: 244 | Page Size: 1 ... Completed!
Dönem: 245 | Page Size: 1 ... Completed!
Dönem: 251 | Page Size: 0 ... Completed!
Dönem: 252 | Page Size: 0 ... Completed!
Dönem: 261 | Page Size: 1 ... Completed!
Dönem: 262 | Page Size: 1 ... Completed!
Dönem: 263 | Page Size: 1 ... Completed!


In [7]:
# Read urls from txt file
with open("urls/arastirma_komisyonlari_urls.txt", "r") as f:
    urls = [url.strip() for url in f.readlines()]

In [8]:
# Add ID to arastirma_komisyonlari DataFrame
arastirma_komisyonlari = pd.read_csv("arastirma_komisyonlari/arastirma_komisyonlari.csv")
arastirma_komisyonlari["id"] = list(range(1, len(arastirma_komisyonlari)+1))
arastirma_komisyonlari.to_csv("arastirma_komisyonlari/arastirma_komisyonlari.csv", index=False)
arastirma_komisyonlari = pd.read_csv("arastirma_komisyonlari/arastirma_komisyonlari.csv")
arastirma_komisyonlari = arastirma_komisyonlari.dropna().reset_index(drop=True)

In [9]:
def get_uyeler(url):
    
    driver.get(url)
    
    uyeler_url = None
    
    sidebar_elements = driver.find_element(By.CLASS_NAME, "blog_right_sidebar").find_element(By.TAG_NAME, "aside").find_element(By.TAG_NAME, "ul").find_elements(By.TAG_NAME, "li")
    for element in sidebar_elements:
        if "üye" in element.text.lower():
            uyeler_url = element.find_element(By.TAG_NAME, "a").get_attribute("href")
            
    if uyeler_url == None:
        return pd.DataFrame()
    else:
        driver.get(uyeler_url)

        table = driver.find_element(By.TAG_NAME, "table")
        rows = table.find_elements(By.TAG_NAME, "tr")
        col_1 = []
        col_2 = []
        col_3 = []

        for row in rows:
            col_1.append(row.find_elements(By.TAG_NAME, "td")[0].text)
            col_2.append(row.find_elements(By.TAG_NAME, "td")[1].text)
            col_3.append(row.find_elements(By.TAG_NAME, "td")[2].text)

        arastirma_komisyonu_uyeler = pd.concat([pd.Series(col_1), pd.Series(col_2), pd.Series(col_3)], axis=1)
        arastirma_komisyonu_uyeler.columns = arastirma_komisyonu_uyeler.iloc[0]
        arastirma_komisyonu_uyeler = arastirma_komisyonu_uyeler.drop(0).reset_index(drop=True)
        return arastirma_komisyonu_uyeler

In [10]:
# Create csv file and headers
pd.DataFrame(columns=["adi_ve_soyadi",
                      "unvan",
                      "secim_bolgesi",
                      "arastirma_komisyon_id"]).to_csv(f"arastirma_komisyonlari/arastirma_komisyonlari_uyeler.csv",
                                                   mode="w", index=False)

In [11]:
for i in range(len(arastirma_komisyonlari)):
    if i % 5 == 0:
        print(f"{i}/{len(arastirma_komisyonlari)} | {(i)/len(arastirma_komisyonlari)*100:.2f}%")
    if "%" not in arastirma_komisyonlari.loc[i, "url"]:
        arastirma_komisyonu_uyeler = get_uyeler(url)
        arastirma_komisyonu_uyeler["arastirma_komisyon_id"] = arastirma_komisyonlari.loc[i, "id"]
        arastirma_komisyonu_uyeler.to_csv(f"arastirma_komisyonlari/arastirma_komisyonlari_uyeler.csv", mode="a+", index=False, header=False)
print(f"{len(arastirma_komisyonlari)}/{len(arastirma_komisyonlari)} | {(len(arastirma_komisyonlari))/len(arastirma_komisyonlari)*100:.2f}%")

0/43 | 0.00%
5/43 | 11.63%
10/43 | 23.26%
15/43 | 34.88%
20/43 | 46.51%
25/43 | 58.14%
30/43 | 69.77%
35/43 | 81.40%
40/43 | 93.02%
43/43 | 100.00%


In [12]:
driver.quit()